In [ ]:
%pip install tk

In [40]:
from deep_translator import GoogleTranslator


In [42]:
translated = GoogleTranslator(source='english', target='hindi').translate_file('Plag.txt')

In [ ]:
print(translated)

In [1]:

A="""Do you know what the Mexicans say about Pacific?
    They say it has no memory.
    That is the place I want to live the rest of my life;
    a warm place with no memory.
    Open up a little Hotel, take my guests out for fishing.
    Charter fishing! They call it."""

B="That is the place I want"

In [1]:
import numpy as np
import re
import nltk
from numpy.linalg import norm

In [2]:
def preProcess(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = text.replace('\n', ' ')
    text = re.sub(r'\[.*?\]|\{.*\}', '', text)

    return text


In [3]:
# Downloading the vocabulary of entire English language
!python -m spacy download en_core_web_md

^C


In [ ]:
# Downloading the vocabulary of entire English language
!python -m spacy download en_core_web_lg

In [ ]:
import spacy

In [ ]:
# Load the language library
nlp = spacy.load('en_core_web_lg')

In [ ]:
# The vector representation of the paragraph A
vecA = nlp(A)
vecA

Do you know what the Mexicans say about Pacific?
    They say it has no memory.
    That is the place I want to live the rest of my life;
    a warm place with no memory.
    Open up a little Hotel, take my guests out for fishing.
    Charter fishing! They call it.

In [ ]:
# The vector representation of the paragraph B
vecB = nlp(B)

In [ ]:
# Vector for the test Text 'B'
X=vecB.vector

In [ ]:
# Finding the index in vecA where cosine similarity is maximum with vecB
n = len(vecB) # Number of tokens in B
ID=0
for i in range(len(vecA)-n):
    Y=vecA[i:i+n].vector

    # Computing the Cosine Similartity between the corresponding 3 words
    cosSim = np.dot(X,Y)/(norm(X)*norm(Y))

    if(cosSim>0.98):
        ID=i
        break


In [ ]:
# The corresponding text in A
vecA[ID:ID+n]


    That is the place I

In [39]:
import re
import nltk
from nltk.util import ngrams, pad_sequence, everygrams
from nltk.lm import MLE, WittenBellInterpolated
from scipy.ndimage import gaussian_filter
import numpy as np
import plotly.graph_objects as go
import spacy

In [353]:
P1 = """Summer is hot for many reasons these include some natural factors and some manmade factors. 
These factors also cause many changes in climate conditions. Although the season is quite dry the kids love it.
Moreover, too much hotness is very bad for certain things and results in many problems.
One problem that it causes in human which is very common is dehydration. It not only causes weakness and dizziness but also can lead to death.
So, to keep the body hydrated we need to drink plenty of water.
Small ponds, rivers, and wells dry up in this season. The groundwater level declines and in some areas conditions like drought occurs."""

P2 = """The foundations of our learning are laid by our schools. 
School is the place that instills in us a sense of responsibility and hardworking spirit. 
We become better versions of ourselves through the rigorous tests that we are put through at schools. 
Each exam, each activity, and each lesson is aimed towards making us learn and grow. 
Right from the beginning, we are taught to be morally responsible for all our actions.
The various subjects introduce us to the world’s diversity and allow us to explore our surroundings with a unique perspective. 
We never know what problems await us when we grow up; schools are the training grounds that prepare us to face the unknown. 
We can overcome any hardship with the right motivation, and schools do that for us. 
We are not only taught to be responsible citizens but also hardworking individuals who strive hard to face the challenges that life throws at us."""

# Remove new lines, tabs and punctuation and lowercase the text
#P1 = P1.lower()
P1 = P1.replace('\n', ' ')
P1 = P1.replace('\t', '')
P1 = re.sub(r'[^\w\s]', '', P1)

#P2 = P2.lower()
P2 = P2.replace('\n', ' ')
P2 = P2.replace('\t', '')
P2 = re.sub(r'[^\w\s]', '', P2)

In [377]:
# PC copied from P1 and P2
PC = """On that weekend at Silverstone, Lewis Hamilton was under pressure for a win as he was lagging by a lot of points.
Sunday's race begins and Max takes the lead. They go through the Wellington Straight and Lewis almost catches up
with Max but he manages to stay ahead. However Hamilton gets a lovely slipstream out of Brooklands and Luffield.
Moreover, too much hotness is very bad for certain things and results in many problems.
One problem that it causes in human which is very common is dehydration. It not only causes weakness and dizziness but also can lead to death.
They get side by side and Max thinks he has the Woodcotes corner but Hamilton turns up on the inside and his front
left wheel touched Max's right rear and sends him barrelling out into the barriers for a massive 51G crash.
We become better versions of ourselves through the rigorous tests that we are put through at schools. 
Each exam, each activity, and each lesson is aimed towards making us learn and grow. 
Right from the beginning, we are taught to be morally responsible for all our actions.
Charles Leclerc manages to surge ahead in front of a slowing Hamilton and takes the lead of the race.
However, he is plagued by engine problems throughout the race and in no way could he keep the charging Lewis behind him.
Lewis soon catches him and overtakes him on the inside at the same Woodcotes corner ironically where he sent Max to his doom.
Eventually, Lewis wins the British Grand Prix in front of his ecstatic fans.
So, to keep the body hydrated we need to drink plenty of water.
Small ponds, rivers, and wells dry up in this season. The groundwater level declines and in some areas conditions like drought occurs."""

# PC = """These factors also cause many changes in climate conditions. Although the season is quite dry the kids love it.
# Moreover, too much hotness is very bad for certain things and results in many problems.
# One problem that it causes in human which is very common is dehydration. It not only causes weakness and dizziness but also can lead to death.
# Each exam, each activity, and each lesson is aimed towards making us learn and grow. 
# Right from the beginning, we are taught to be morally responsible for all our actions.
# The various subjects introduce us to the world’s diversity and allow us to explore our surroundings with a unique perspective. 
# We never know what problems await us when we grow up; schools are the training grounds that prepare us to face the unknown."""

#PC = PC.lower()
PC = PC.replace('\n', ' ')
PC = PC.replace('\t', '')
PC = re.sub(r'[^\w\s]', '', PC)


In [408]:
reference = {0:'P1',1:'P2'}

In [378]:
# Load the language library
nlp = spacy.load('en_core_web_lg')

In [379]:
vP1 = nlp(P1)
vP2 = nlp(P2)

vP = nlp(P1+' '+P2)

In [380]:
n=5

In [381]:
words = [w.text for w in vP]
training_data = list(pad_sequence(words, n, 
                                  pad_left=True, 
                                  left_pad_symbol="<s>"))

In [382]:
# Generate n-grams from the training data
ngrams = list(everygrams(training_data, max_len=n))
# Build n-gram language model
model = WittenBellInterpolated(n)
model.fit([ngrams],vocabulary_text=training_data)

In [383]:
vPC = nlp(PC)
wordsPlag = [w.text for w in vPC]

In [384]:
test_data = list(pad_sequence(wordsPlag, n, 
                                  pad_left=True, 
                                  left_pad_symbol="<s>"))

In [417]:
score=[]
#source = []
# Generate score
for i,ele in enumerate(test_data[n-1:]):
    s = model.score(ele,test_data[i:i+n-1])
    score.append(s)

# Convert to numpy array
score_np=np.array(score)
#source_np=np.array(source)

In [418]:
# Set height and width of the graph
width=12
height=np.ceil(len(score_np)/width).astype("int32")

In [419]:
# Copy the score_np to a new array with shape (height,width)
a=np.zeros(width*height)
sourceLabel = ['NA']*(width*height) # source label to be used to identify original source of plagiarised portions
a[:len(score_np)] = score_np
sourceLabel = np.array(sourceLabel)
diff = len(a) - len(score_np)

In [420]:
# Apply gaussian filter to the array
a = gaussian_filter(a, sigma=1.0)
# a=a*1.093 # To make the probabilties better (which have been lowered because of the gaussian filter)
# Reshape to fit rectangular shape
a = a.reshape(-1,width)

In [421]:
# format labels
labels = [" ".join(vPC[i:i+width].text.split()) for i in range(0, len(vPC), width)]
labels_individual = [x.split() for x in labels]
labels_individual[-1] += [""]*diff
labels = [f"{x:60.60}" for x in labels]

In [422]:
a = a.reshape(-1)

plagScore=np.sum(a[0:a.shape[0]-diff])/(a.shape[0]-diff) # Avg. Score (excluding the padding and blank text)
plagPercentage=plagScore*100
plagPercentage

36.170554774500744

In [423]:
# The entire corpus of Original Documents
corpus = []
corpus.append(vP1)
corpus.append(vP2)


In [424]:
# Dict to store which plagiarised portion belongs to which Document
plag_dict = {}

In [425]:
def check_plagiarised(j,vecO,vecP,id,fin,plag_dict):
    for k in range(len(vecO)-3):
        # X is Vector of the plagiarized text present between indexes id and fin
        X=vecP[id:fin+1]
        # Y is Vector of the original text present between indexes j and j+l
        Y=vecO[k:k+fin-id+1]
        
        #Computing cosine similarity
        sim = X.similarity(Y)

        # If the cosine similarity is greater than 0.997, then we get a match from original text
        if(sim>0.997):
            if(j in plag_dict):
                a=[id,fin]
                b=[k,k+fin-id+1]
                plag_dict[j].append([a,b])
            else:
                a=[id,fin]
                b=[k,k+fin-id+1]
                plag_dict[j] = [[a,b]]
            break
    
    return plag_dict

In [426]:
plID=[] 
# The index of the tokens which are plagiarized (index wrt to plagiarized text)
for i in range(len(a)-2):
    if(a[i-1]>0.58 or a[i]>0.62 or a[i+1]>0.58):
        #pl.append(testing_data[i+n-1])
        plID.append(i)

In [427]:
# To note the fact that plagiarism can be scattered
# Anytime the indexes difference > 4, then it means those two plagiarised words came from different parts of original doc (Flaw exists but we'll improve)
jumpStates=[]
jumpStates.append(plID[0])
for i in range(len(plID)):
    if(plID[i]-plID[i-1]>4):
        jumpStates.append(plID[i-1])
        jumpStates.append(plID[i])

jumpStates.append(plID[-1])

In [428]:
# Fill plag_dict with the plagiarised portions
for i in range(0,len(jumpStates),2):
    id=jumpStates[i]
    fin=jumpStates[i+1]

    # Checking which document the plagiarised portion belongs to
    for j,vec in enumerate(corpus):
        plag_dict=check_plagiarised(j,corpus[j],vPC,id,fin,plag_dict)
        

In [429]:
# Iterate over plag_dict
for i in plag_dict:
    v = plag_dict[i]
    for j in v:
        # Extracting the plagiarised portions
        st = j[0][0]
        en = j[0][1]

        # Labelling those portions
        sourceLabel[st:en+1] = reference[i]*(en-st+1)


In [430]:
sourceLabel = sourceLabel.reshape(-1,width)

In [431]:
# create heatmap of the Plagiarized Text
a=a.reshape(-1,width)
fig = go.Figure(data=go.Heatmap(
                z=a, x0=0, dx=1,
                #y=labels, 
                zmin=0, zmax=1,
                customdata=sourceLabel,
                hovertemplate='%{customdata} <br><b>Score:%{z:.3f}<extra></extra>',
                text=labels_individual,
                texttemplate='%{text}',
                textfont={"size":8},
                colorscale='reds'
                ))
fig.update_layout({"height":height*25, "width":1000, "font":{"family":"sans-serif"}})
fig['layout']['yaxis']['autorange'] = "reversed"

In [432]:
fig.show() # Hover over the plagiarised portions to see the source